In [1]:
# Загрузка необходимых модулей для анализа данных и формирования выборки

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandasql as ps
import regex as re
from datetime import datetime as dt

from transliterate import translit

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Функция транслитерации и удаление ненужных симфолов из названий столбцов

def del_trash(x):
    x1 = re.sub("[^A-Za-z]",' ',translit(x,'ru',reversed=True)).lower()
    x2 = re.sub(r'\s+',' ',x1)
    return x2.replace(' ','_')

### 1. Формирование выборки на основе ИНН, даты открытия, закрытия, и блокировки с целью формирования таргета по Бизнес-задаче

In [3]:
# Загрузка данных об открытых счетах и добровольного закрытия. Проверка типов колонок, размера выборки

df_open = pd.read_excel('Открытые рс.xlsx',\
                        parse_dates=True)

df_open.columns = [del_trash(i) for i in list(df_open.columns)]
print(df_open.dtypes)
print()
print('Shape: ', df_open.shape)
df_open.head()

inn                       object
data_otkrytija    datetime64[ns]
data_zakrytija            object
dtype: object

Shape:  (246933, 3)


,inn,data_otkrytija,data_zakrytija
0,KTIZMVIQGVVW,2019-09-25,2019-12-18 00:00:00
1,SRCUMJUBNXUM,2019-04-29,2019-06-26 00:00:00
2,KZAGDPVOXANB,2019-06-24,2019-08-29 00:00:00
3,CYCMZJKVLKAZ,2019-10-10,2020-01-30 00:00:00
4,QLQYBUPRBBSJ,2019-10-10,2019-12-24 00:00:00


In [4]:
# Приведение к форматам и удаление дубликатов

date_cols = ['data_otkrytija','data_zakrytija']

df_open[date_cols] = df_open[date_cols].apply(lambda x: pd.to_datetime(x, errors = 'coerce'))
df_open['year_mon_open'] = df_open['data_otkrytija'].dt.strftime('%Y-%m')
df_open['quarter_open'] = pd.PeriodIndex(pd.to_datetime(df_open['data_otkrytija']), freq='Q').astype(str)
df_open['data_otkrytija'] = df_open['data_otkrytija'].dt.strftime('%Y-%m-%d')

df_open = df_open.drop_duplicates()
print('Shape after drop duplicates: ', df_open.shape)
print('Nunique of inn: ', df_open.inn.nunique())
df_open.head(3)

Shape after drop duplicates:  (246468, 5)
Nunique of inn:  218984


,inn,data_otkrytija,data_zakrytija,year_mon_open,quarter_open
0,KTIZMVIQGVVW,2019-09-25,2019-12-18,2019-09,2019Q3
1,SRCUMJUBNXUM,2019-04-29,2019-06-26,2019-04,2019Q2
2,KZAGDPVOXANB,2019-06-24,2019-08-29,2019-06,2019Q2


In [5]:
# Проверка данных на дважды и более открытые счета без даты закрытия

df_check = df_open.groupby('inn').count()
strange_inn = list(df_check[(df_check['data_otkrytija']-df_check['data_zakrytija'])>=2].index)

df_check = df_open[df_open['inn'].isin(strange_inn)].\
sort_values(by=['inn','data_otkrytija'],ascending=True).drop_duplicates()

print(df_check.shape)
df_check[['inn','data_otkrytija','data_zakrytija']].head(8)

(5706, 5)


,inn,data_otkrytija,data_zakrytija
54955,AAAICRJTBA,2018-08-01,NaT
233604,AAAICRJTBA,2019-10-11,NaT
20121,AASJXTWMFL,2019-07-18,2019-10-18
217762,AASJXTWMFL,2019-07-18,NaT
218093,AASJXTWMFL,2019-11-28,NaT
16067,AAXRATRWNIAO,2019-04-23,2019-08-07
183458,AAXRATRWNIAO,2019-04-23,NaT
185499,AAXRATRWNIAO,2019-08-28,NaT


In [6]:
df_open[df_open.inn=='AASJXTWMFL'].drop_duplicates(subset=['inn','data_otkrytija'],keep='first')

,inn,data_otkrytija,data_zakrytija,year_mon_open,quarter_open
20121,AASJXTWMFL,2019-07-18,2019-10-18,2019-07,2019Q3
218093,AASJXTWMFL,2019-11-28,NaT,2019-11,2019Q4


In [7]:
df_open = df_open.drop_duplicates(subset=['inn','data_otkrytija'],keep='first')
print(df_open.shape)

(221760, 5)


In [8]:
# Проверка, есть ли наблюдения у которых дата открытия позже даты закрытия

df_open[df_open.data_zakrytija<df_open.data_otkrytija]

,inn,data_otkrytija,data_zakrytija,year_mon_open,quarter_open


In [9]:
# Проверка на количество пропущенных значений

df_open.isna().sum()

inn                    0
data_otkrytija         0
data_zakrytija    189614
year_mon_open          0
quarter_open           0
dtype: int64

- Вывод: В витрине существуют клиенты с открытием счета более 1 раза, без закрытия хотя бы по одному. 
* Вопрос: Может ли один ИНН открывать счет дважды или возможно его блокировали после первого раза и он открыл после?
* Ответ: Проверим после присоединения данных о блокировке!

In [10]:
# Загрузка данных о блокировке счетов. Проверка типов колонок, размера выборки, количество уникальных ИНН

df_block = pd.read_excel('Блокировки.xlsx')

df_block.columns = [del_trash(i) for i in list(df_block.columns)]
print(df_block.dtypes)
print()
print('Shape: ', df_block.shape)
df_block.head()

inn                        object
data_blokirovki    datetime64[ns]
dtype: object

Shape:  (31214, 2)


,inn,data_blokirovki
0,UWJVWILNVB,2018-06-18
1,DIQZQQEMLN,2018-06-18
2,CECYYCSEAKLP,2018-06-19
3,OCMHZOYWACDN,2018-06-19
4,RDFCTRXYFYHM,2018-06-19


In [11]:
# Приведение к форматам и удаление дубликатов

df_block['year_mon_block'] = pd.to_datetime(df_block.data_blokirovki).dt.strftime('%Y-%m')
df_block['data_blokirovki'] = pd.to_datetime(df_block.data_blokirovki).dt.strftime('%Y-%m-%d')

df_block = df_block.drop_duplicates()
print('Shape after drop duplicates: ', df_block.shape)
print('Nunique: ', df_block.inn.nunique())
df_block.head(2)

Shape after drop duplicates:  (31201, 3)
Nunique:  30096


,inn,data_blokirovki,year_mon_block
0,UWJVWILNVB,2018-06-18,2018-06
1,DIQZQQEMLN,2018-06-18,2018-06


In [12]:
# Проверка на количество пропущеных значений

df_block.isna().sum()

inn                0
data_blokirovki    0
year_mon_block     0
dtype: int64

In [13]:
# Проверка типов данных перед склейкой данных об открытых счетах и данных об блокировке счетов

df_open.dtypes, df_block.dtypes

(inn                       object
 data_otkrytija            object
 data_zakrytija    datetime64[ns]
 year_mon_open             object
 quarter_open              object
 dtype: object,
 inn                object
 data_blokirovki    object
 year_mon_block     object
 dtype: object)

In [14]:
# Соединяем выборки ИНН открытых счетов и ИНН блокировок через left join, с условием на то, 
# что дата блокировки должна быть больше либо равна даты открытия счета или должна оставаться пусткой, так как
# блокировки не происходило

query = """
SELECT a.*, 
    b.data_blokirovki, 
    b.year_mon_block
FROM df_open a
LEFT JOIN df_block b
ON a.inn=b.inn and a.data_otkrytija<=b.data_blokirovki
ORDER BY a.data_otkrytija
"""

df_join = ps.sqldf(query,locals())

print(df_join.shape)
df_join.head()

(222776, 7)


,inn,data_otkrytija,data_zakrytija,year_mon_open,quarter_open,data_blokirovki,year_mon_block
0,VWNMBTGADCEF,2018-06-18,2019-02-08 00:00:00.000000,2018-06,2018Q2,None,None
1,EXSWZGFDABIU,2018-06-18,2018-06-21 00:00:00.000000,2018-06,2018Q2,None,None
2,IUOMTGZHHT,2018-06-18,2019-01-09 00:00:00.000000,2018-06,2018Q2,None,None
3,EUJAITITRDJL,2018-06-18,None,2018-06,2018Q2,None,None
4,TPNQVLMRXW,2018-06-18,None,2018-06,2018Q2,None,None


In [15]:
# Проверка на количество наблюдений с датой открытия меньшей либо равной дате блокировки
# Проверка на количество наблюдений с датой открытия большей даты блокировки

print(df_join[df_join.data_otkrytija<=df_join.data_blokirovki].shape, ';', 
df_join[df_join.data_otkrytija>df_join.data_blokirovki].shape)

(23540, 7) ; (0, 7)


In [16]:
# Приведение дат к соответствующим форматам

date_cols = ['data_otkrytija','data_zakrytija','data_blokirovki']

df_join[date_cols] = df_join[date_cols].apply(lambda x: pd.to_datetime(x,errors = 'coerce'))
print(df_join.dtypes)

inn                        object
data_otkrytija     datetime64[ns]
data_zakrytija     datetime64[ns]
year_mon_open              object
quarter_open               object
data_blokirovki    datetime64[ns]
year_mon_block             object
dtype: object


In [17]:
# Проверка данных на дважды и более открытые счета без даты закрытия с датой блокировки

df_check = df_join.groupby('inn').count()
strange_inn = list(df_check[(df_check['data_otkrytija']-df_check['data_zakrytija'])>=2].index)

df_check = df_join[df_join['inn'].isin(strange_inn)].\
sort_values(by=['inn','data_otkrytija'],ascending=True)

print(df_check.shape)
df_check[df_check.data_zakrytija.notna()&df_check.data_blokirovki.notna()][['inn']+date_cols].head(20)

(3524, 7)


,inn,data_otkrytija,data_zakrytija,data_blokirovki
12494,ARLKKRHUIH,2018-08-15,2019-04-18,2019-02-27
12495,ARLKKRHUIH,2018-08-15,2019-04-18,2019-03-05
108257,BUGZKXXAMJ,2019-07-11,2019-09-18,2019-09-09
108258,BUGZKXXAMJ,2019-07-11,2019-09-18,2019-10-01
41488,BXCKDNIDDG,2018-12-19,2019-01-30,2019-01-21
41489,BXCKDNIDDG,2018-12-19,2019-01-30,2019-01-30
131814,ENJOXKDKXW,2019-08-27,2019-10-17,2019-09-30
131815,ENJOXKDKXW,2019-08-27,2019-10-17,2019-10-18
51371,ESYKAUXEWU,2019-02-06,2019-03-20,2019-02-27
51372,ESYKAUXEWU,2019-02-06,2019-03-20,2019-02-28


In [18]:
df_check[df_check.inn.isin(['ARLKKRHUIH','BUGZKXXAMJ','BXCKDNIDDG'])][['inn']+date_cols]

,inn,data_otkrytija,data_zakrytija,data_blokirovki
12494,ARLKKRHUIH,2018-08-15,2019-04-18,2019-02-27
12495,ARLKKRHUIH,2018-08-15,2019-04-18,2019-03-05
13158,ARLKKRHUIH,2018-08-17,NaT,2019-02-27
13159,ARLKKRHUIH,2018-08-17,NaT,2019-03-05
99198,BUGZKXXAMJ,2019-06-21,NaT,2019-09-09
99199,BUGZKXXAMJ,2019-06-21,NaT,2019-10-01
108257,BUGZKXXAMJ,2019-07-11,2019-09-18,2019-09-09
108258,BUGZKXXAMJ,2019-07-11,2019-09-18,2019-10-01
41319,BXCKDNIDDG,2018-12-18,NaT,2019-01-21
41320,BXCKDNIDDG,2018-12-18,NaT,2019-01-30


In [19]:
df_check.inn.nunique()

1647

In [20]:
# Т.е. при разной дате открытия счета, дата блокировки по ИНН происходила +- несколько дней. 
# Значит целесообразно рассматривать блокировки на уровне ИНН
# Таким образом удаляем дубликаты по связке ИНН - дата открытия с сохранением последней даты, 
# к которой и присоединяется последняя дата блокировки

df_join = df_join.drop_duplicates(subset=['inn','data_blokirovki'],keep='last').\
drop_duplicates(subset=['inn','data_otkrytija'],keep='last')

In [21]:
df_join[df_join.inn.isin(['ARLKKRHUIH','BUGZKXXAMJ','BXCKDNIDDG'])].\
sort_values(by=['inn','data_otkrytija'],ascending=True)[['inn']+date_cols]

,inn,data_otkrytija,data_zakrytija,data_blokirovki
13159,ARLKKRHUIH,2018-08-17,NaT,2019-03-05
108258,BUGZKXXAMJ,2019-07-11,2019-09-18,2019-10-01
41489,BXCKDNIDDG,2018-12-19,2019-01-30,2019-01-30


In [22]:
# Проверка данных на дважды и более открытые счета без даты закрытия с датой блокировки

df_check = df_join.groupby('inn').count()
strange_inn = list(df_check[(df_check['data_otkrytija']-df_check['data_zakrytija'])>=2].index)

df_check = df_join[df_join['inn'].isin(strange_inn)].\
sort_values(by=['inn','data_otkrytija'],ascending=True)

print(df_check.shape)
df_check[df_check.data_blokirovki.notna()][['inn']+date_cols].head(10)

(4, 7)


,inn,data_otkrytija,data_zakrytija,data_blokirovki
90502,HYDOCZHAKULQ,2019-05-30,NaT,2019-09-03
200772,ULORZTNOXO,2019-12-05,NaT,2019-12-09
204815,ULORZTNOXO,2019-12-11,NaT,2020-01-24


In [23]:
df_join.shape

(218992, 7)

In [24]:
df_join.shape[0]-df_join.inn.nunique()

8

In [25]:
double_inn = pd.DataFrame(df_join.groupby('inn').count()).\
sort_values('data_otkrytija',ascending=False)[:4].index.tolist()

In [26]:
len(double_inn), double_inn

(4, ['HYDOCZHAKULQ', 'ZCRMIKGJRE', 'HZIOMXGDPN', 'RMUFIWAWFP'])

In [27]:
df_join[df_join.inn.isin(double_inn)].\
sort_values(by=['inn','data_otkrytija'],ascending=True)[['inn']+date_cols]

,inn,data_otkrytija,data_zakrytija,data_blokirovki
90502,HYDOCZHAKULQ,2019-05-30,NaT,2019-09-03
147150,HYDOCZHAKULQ,2019-09-19,NaT,NaT
48551,HZIOMXGDPN,2019-01-28,2019-04-24,2019-04-22
159865,HZIOMXGDPN,2019-10-08,NaT,2020-02-04
114304,RMUFIWAWFP,2019-07-24,NaT,2019-08-22
134566,RMUFIWAWFP,2019-08-30,2019-09-09,NaT
70332,ZCRMIKGJRE,2019-04-01,2019-05-15,2019-04-15
76394,ZCRMIKGJRE,2019-04-17,2019-04-26,2019-04-19


* Логика клиента следующая: 1) Он открывает счет, его блочат, он его закрывает. 2) Открывает снова, и либо данные не вызрели, либо он смог оспорить блокировку. Здесь из 4х инн двое были окончательно заблочены, один закрылся сам (возможно снова махинация и его заблокируют) и по одному нет вызревания.

* Вывод: Так как мы не знаем номера счетов, лучше от таких инн отказаться, для избежания проблемы лика

In [28]:
df_join = df_join[~df_join.inn.isin(double_inn)]

In [29]:
# Размечаем наши данные! Те клиенты, у кого разница между датой блокировки и датой открытия счета от 1-6 месяцев 
# являются целевым классом = 1, все остальные = 0
df_join['diff_date_m'] = ((df_join.data_blokirovki - df_join.data_otkrytija)/ np.timedelta64(1, 'M')).fillna(999).astype('int')

# Фильтруем данные где дата открытия была позже даты блокировки
df_join = df_join[df_join.diff_date_m>=0]
df_join['target'] = np.where((df_join.diff_date_m.isin(list(np.arange(0,7)))),1,0)

In [30]:
# Проверка на вхождения неприятных ИНН ['HYDOCZHAKULQ', 'ZCRMIKGJRE', 'HZIOMXGDPN', 'RMUFIWAWFP']

df_join[df_join.inn.isin(double_inn)][['inn','data_otkrytija','data_blokirovki','target']].sort_values(by=['inn','data_otkrytija'],ascending=True)

,inn,data_otkrytija,data_blokirovki,target


In [31]:
df_join.data_otkrytija.max(), df_join.data_blokirovki.max()

(Timestamp('2019-12-31 00:00:00'), Timestamp('2020-02-05 00:00:00'))

In [32]:
# Фильтруем по ИНН, в которых дата открытия позже даты даты блокировки. Так как нам интересно поведение клиента перед тем
# как его заблокировали, а не после того, как он возможно обжаловал блокировку и открыл счет снова.

drop_df = df_join[(df_join.data_blokirovki<df_join.data_otkrytija)]
df_razmetka = df_join[~df_join.index.isin(drop_df.index)]
df_razmetka.shape,df_razmetka.target.sum(),df_razmetka.target.mean()

((218984, 9), 20627, 0.09419409637233771)

### 2. Формирование пространства признаков на дату поступления и загрузки заявки на открытие РКО

In [33]:
# Загрузка данных об поданых заявках на открытие РКО и типа организации. Проверка типов колонок, размера выборки, количество уникальных ИНН

df_req = pd.read_excel('Заявки.xlsx',\
                       parse_dates=True)

df_req.columns = [del_trash(i) for i in list(df_req.columns)]
print(df_req.dtypes)
print()
print('Shape: ', df_req.shape)
print('Nunique of inn: ', df_req.inn.nunique())
df_req.head()

idinquiry                    int64
inn                         object
data_zajavki        datetime64[ns]
tip_organizatsii            object
dtype: object

Shape:  (387235, 4)
Nunique of inn:  243851


,idinquiry,inn,data_zajavki,tip_organizatsii
0,149105,BYKKBSYFTS,2018-06-18 03:37:00.000000,ЮЛ
1,149106,MIWYERAPTINW,2018-06-18 03:46:00.000000,ИП
2,149107,MIWYERAPTINW,2018-06-18 03:49:00.000000,ИП
3,149109,RXDNYNCXKXRF,2018-06-18 04:12:00.000000,ИП
4,149110,KBTUOUECTSKX,2018-06-18 04:19:59.999999,ИП


In [34]:
# Создание необходимых дат для последующего анализа, упорядочивание по времени заявки по возрастанию

df_req = df_req.rename(columns={'data_zajavki':'time_zajavki'}).sort_values('time_zajavki',ascending=True)

df_req['data_zajavki'] = pd.to_datetime(df_req['time_zajavki']).dt.strftime('%Y-%m-%d')
df_req['year_mon_zajavki'] = pd.to_datetime(df_req['time_zajavki']).dt.strftime('%Y-%m')
df_req['quarter_zajavki'] = pd.PeriodIndex(pd.to_datetime(df_req['time_zajavki']), freq='Q').astype(str)
df_req['year_zajavki'] = pd.to_datetime(df_req['time_zajavki']).dt.strftime('%Y')

df_req.head()

,idinquiry,inn,time_zajavki,tip_organizatsii,data_zajavki,year_mon_zajavki,quarter_zajavki,year_zajavki
0,149105,BYKKBSYFTS,2018-06-18 03:37:00.000000,ЮЛ,2018-06-18,2018-06,2018Q2,2018
1,149106,MIWYERAPTINW,2018-06-18 03:46:00.000000,ИП,2018-06-18,2018-06,2018Q2,2018
2,149107,MIWYERAPTINW,2018-06-18 03:49:00.000000,ИП,2018-06-18,2018-06,2018Q2,2018
3,149109,RXDNYNCXKXRF,2018-06-18 04:12:00.000000,ИП,2018-06-18,2018-06,2018Q2,2018
4,149110,KBTUOUECTSKX,2018-06-18 04:19:59.999999,ИП,2018-06-18,2018-06,2018Q2,2018


In [35]:
# Проверка соотношения ЮЛ и ИП

df_req.groupby('tip_organizatsii')[['idinquiry','inn']].count()

,idinquiry,inn
tip_organizatsii,,
ИП,192583,192583
ЮЛ,194652,194652


In [36]:
# Проверка на количество пропущенных значений

df_req.isna().sum()

idinquiry           0
inn                 0
time_zajavki        0
tip_organizatsii    0
data_zajavki        0
year_mon_zajavki    0
quarter_zajavki     0
year_zajavki        0
dtype: int64

In [37]:
# Для избежания разницы в минутах подачи по дате подачи заявкок, 
# удаляем дубликаты по связке ИНН и дата в формате "Год-месяц-число" (так как за день информация по нему новой не будет), 
# с сортировкой по дате подачи заявки

df_req = df_req.sort_values(by='time_zajavki',ascending=True).\
sort_values(by='time_zajavki',ascending=True).\
drop_duplicates(subset=['inn','data_zajavki'],keep='last')

# Проверка размера выборки и количества уникальных ИНН в выборке
print('Shape after drop duplicates: ', df_req.shape, '\n',
      'Nunique of inn: ', df_req.inn.nunique())

Shape after drop duplicates:  (324852, 8) 
 Nunique of inn:  243851


In [38]:
# Загрузка сведений из СПАРК на момент загрузки заявки на открытие РКО. Проверка типа колонки даты, размера выборки, количество уникальных ИНН

spark = pd.read_csv('СПАРК.csv',\
                   parse_dates=True)
spark.columns = [del_trash(i) for i in list(spark.columns)]
print(spark.data_zagruzki_zajavki.dtype)
print()
print('Shape: ', spark.shape)
print('Nunique of inn: ', spark.inn.nunique())
spark.head()

object

Shape:  (145507, 71)
Nunique of inn:  100000


,inn,data_zagruzki_zajavki,kolichestvo_kompanij_s_analogichnym_direktorom_v_tom_zhe_regione,kompanija_vhodit_v_juridicheskie_litsa_v_sostav_ispolnitel_nyh_organov_kotoryh_vhodjat_diskvalifitsirovannye_litsa,kolichestvo_kompanij_zaregistrirovannyh_na_adrese_registratsii_organizatsii_po_dannym_sajta_fns,kolichestvo_vidov_dejatel_nosti_u_klienta,kod_osnovnogo_okved,sub_ekt_mestonahozhdenija,razmer_ustavnogo_kapital_jul,chistaja_pribyl_ili_ubytok_kompanii,...,_summiruem_razmery_dolej_uchreditelej_inostrannyh_jul_iz_sektsii,_srok_registratsii_klienta,_kodokved,_prversokved,_a_upravljajuschaja_organizatsija_berem_dannye_iz_sektsii_svuprorg,_b_gendirektor_fl_,_uchrediteli_i_ih_izmenenie,_uchrediteli_inostrannye_kompanii,_uchrediteli_fizlitsa,istorija_smeny_soouchreditelej
0,JVGAVXQOMXDL,2018-09-24 14:04:47.397,0.0,Нет,NaN,32.0,81.29,GN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,VYSZNCKGGM,2018-11-07 09:28:42.913,2.0,Нет,2.0,13.0,46.72,YS,10000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ЮЛ: 14.06.2018 0:00:00; ЮЛ: 18.02.2016 0:00:00
2,OAUNBEVXYM,2018-12-19 05:19:50.927,2.0,Нет,12.0,9.0,46.73,ZY,10000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ЮЛ: 09.10.2018 0:00:00
3,POTUIFVYMPZL,2019-10-25 17:36:45.310,0.0,Нет,NaN,1.0,47.59.1,XT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KVVWUJYQTVFV,2019-02-04 15:10:23.197,0.0,Нет,NaN,1.0,68.20.2,VM,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# Приведение даты к соответствующему формату, а также создание дополнительных столбцов с определенным форматом даты
# Упорядочивание по возрастанию на дату загрузки заявки

spark = spark.rename(columns={'data_zagruzki_zajavki':'time_zagruzki_zajavki'}).sort_values(by='time_zagruzki_zajavki',ascending=True)

spark['time_zagruzki_zajavki'] = pd.to_datetime(spark['time_zagruzki_zajavki'])
spark['data_zagruzki_zajavki'] = pd.to_datetime(spark['time_zagruzki_zajavki']).dt.strftime('%Y-%m-%d')
spark['year_mon_zagruzki_zajavki'] = pd.to_datetime(spark['time_zagruzki_zajavki']).dt.strftime('%Y-%m')

print(spark[['inn','time_zagruzki_zajavki','data_zagruzki_zajavki','year_mon_zagruzki_zajavki']].dtypes)
spark.head()

inn                                  object
time_zagruzki_zajavki        datetime64[ns]
data_zagruzki_zajavki                object
year_mon_zagruzki_zajavki            object
dtype: object


,inn,time_zagruzki_zajavki,kolichestvo_kompanij_s_analogichnym_direktorom_v_tom_zhe_regione,kompanija_vhodit_v_juridicheskie_litsa_v_sostav_ispolnitel_nyh_organov_kotoryh_vhodjat_diskvalifitsirovannye_litsa,kolichestvo_kompanij_zaregistrirovannyh_na_adrese_registratsii_organizatsii_po_dannym_sajta_fns,kolichestvo_vidov_dejatel_nosti_u_klienta,kod_osnovnogo_okved,sub_ekt_mestonahozhdenija,razmer_ustavnogo_kapital_jul,chistaja_pribyl_ili_ubytok_kompanii,...,_kodokved,_prversokved,_a_upravljajuschaja_organizatsija_berem_dannye_iz_sektsii_svuprorg,_b_gendirektor_fl_,_uchrediteli_i_ih_izmenenie,_uchrediteli_inostrannye_kompanii,_uchrediteli_fizlitsa,istorija_smeny_soouchreditelej,data_zagruzki_zajavki,year_mon_zagruzki_zajavki
54576,OKVBXLMESK,2018-01-09 05:22:21.170,1.0,Нет,1.0,9.0,46.19,RT,80000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ЮЛ: 18.12.2017 0:00:00,2018-01-09,2018-01
142029,GAGOSTYFBT,2018-01-09 05:29:08.950,NaN,NaN,NaN,NaN,NaN,SM,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-09,2018-01
79805,APAOYKOVHCRH,2018-01-09 12:58:44.493,0.0,Нет,NaN,7.0,70.22,FI,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-09,2018-01
16867,XPARYIKHYX,2018-01-09 14:00:37.050,1.0,Нет,33.0,14.0,52.24,EG,10000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ЮЛ: 27.12.2017 0:00:00,2018-01-09,2018-01
27893,AWXBKPFQUCBM,2018-01-09 16:10:15.837,0.0,Нет,NaN,2.0,95.2,QI,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-09,2018-01


In [40]:
# Проверка на количество пропусков по ИНН и дате загрузки заявки

spark[['inn','time_zagruzki_zajavki']].isna().sum()

inn                      0
time_zagruzki_zajavki    0
dtype: int64

In [41]:
# Проверка на максимальный размер выборки по входящим ИНН из СПАРК в данные о заявлении на открытие РКО

df_req[df_req.inn.isin(spark.inn.unique().tolist())].shape[0]

124017

In [42]:
# Фильтруем витрину с информацией по заявках на открытие РКО по ИНН, у которых есть информация из СПАРК

df_req_filter_inn = df_req[df_req.inn.isin(spark.inn.unique().tolist())]

print('Shape: ', df_req_filter_inn.shape)
print('Nunique of inn: ', df_req_filter_inn.inn.nunique())
df_req_filter_inn.head(3)

Shape:  (124017, 8)
Nunique of inn:  100000


,idinquiry,inn,time_zajavki,tip_organizatsii,data_zajavki,year_mon_zajavki,quarter_zajavki,year_zajavki
0,149105,BYKKBSYFTS,2018-06-18 03:37:00,ЮЛ,2018-06-18,2018-06,2018Q2,2018
2,149107,MIWYERAPTINW,2018-06-18 03:49:00,ИП,2018-06-18,2018-06,2018Q2,2018
3,149109,RXDNYNCXKXRF,2018-06-18 04:12:00,ИП,2018-06-18,2018-06,2018Q2,2018


In [43]:
# Соединяем данные из СПАРКА с данными о заявках на открытие счета, уже профильтрованных по ИНН у которых есть информация,
# при этом ставим фильтр на то, чтобы даты в формате ("Год-Месяц-Число") загрузка заявки 
# была больше либо равной дате создания заявки с удалением дубликатов по id заявки, ИНН и дате подачи заявки

query = """
SELECT a.idinquiry,
    a.time_zajavki,
    a.tip_organizatsii,
    a.data_zajavki,
    a.year_mon_zajavki,
    a.quarter_zajavki,
    a.year_zajavki,
    b.*
FROM df_req_filter_inn a
LEFT JOIN spark b
ON a.inn=b.inn and a.data_zajavki<=b.data_zagruzki_zajavki
WHERE b.time_zagruzki_zajavki IS NOT NULL
ORDER BY a.time_zajavki
"""

df_features = ps.sqldf(query,locals())
df_features.shape

(175419, 80)

In [44]:
df_features = df_features.drop_duplicates(subset=['idinquiry','inn'],keep='last')
print(df_features.shape, df_features.inn.nunique())
df_features.head()

(123927, 80) 99921


,idinquiry,time_zajavki,tip_organizatsii,data_zajavki,year_mon_zajavki,quarter_zajavki,year_zajavki,inn,time_zagruzki_zajavki,kolichestvo_kompanij_s_analogichnym_direktorom_v_tom_zhe_regione,...,_kodokved,_prversokved,_a_upravljajuschaja_organizatsija_berem_dannye_iz_sektsii_svuprorg,_b_gendirektor_fl_,_uchrediteli_i_ih_izmenenie,_uchrediteli_inostrannye_kompanii,_uchrediteli_fizlitsa,istorija_smeny_soouchreditelej,data_zagruzki_zajavki,year_mon_zagruzki_zajavki
0,149105,2018-06-18 03:37:00.000000,ЮЛ,2018-06-18,2018-06,2018Q2,2018,BYKKBSYFTS,2018-06-18 03:34:03.993000,2.0,...,None,None,None,None,None,None,None,ЮЛ: 08.06.2018 0:00:00,2018-06-18,2018-06
2,149107,2018-06-18 03:49:00.000000,ИП,2018-06-18,2018-06,2018Q2,2018,MIWYERAPTINW,2018-06-18 03:46:46.243000,0.0,...,None,None,None,None,None,None,None,None,2018-06-18,2018-06
3,149109,2018-06-18 04:12:00.000000,ИП,2018-06-18,2018-06,2018Q2,2018,RXDNYNCXKXRF,2018-06-18 04:09:41.117000,0.0,...,None,None,None,None,None,None,None,None,2018-06-18,2018-06
4,149112,2018-06-18 04:56:00.000001,ЮЛ,2018-06-18,2018-06,2018Q2,2018,EGULYSKZPC,2018-06-18 04:53:47.150000,1.0,...,None,None,None,None,None,None,None,ЮЛ: 14.06.2018 0:00:00,2018-06-18,2018-06
10,149113,2018-06-18 05:19:00.000000,ИП,2018-06-18,2018-06,2018Q2,2018,RQAOYTOOVXCE,2018-08-06 06:07:58.430000,0.0,...,None,None,None,None,None,None,None,None,2018-08-06,2018-08


In [45]:
df_features.groupby('inn').count().sort_values(by='idinquiry',ascending=False)[['idinquiry']].head(5)

,idinquiry
inn,
BQQAMCUYXH,13
BYDTYIPRAJKN,11
QSLWIOCSGG,11
VDAGPSZFKS,11
ZOTUSBBWIS,10


In [46]:
check_inn = ['ZOTUSBBWIS']
df_features[df_features.inn.isin(check_inn)][['idinquiry','inn','time_zajavki','time_zagruzki_zajavki']]

,idinquiry,inn,time_zajavki,time_zagruzki_zajavki
80799,240163,ZOTUSBBWIS,2019-01-23 11:08:00.000000,2019-10-07 14:24:51.210000
89388,249405,ZOTUSBBWIS,2019-02-07 17:12:00.000000,2019-10-07 14:24:51.210000
109894,269062,ZOTUSBBWIS,2019-03-11 16:46:00.000000,2019-10-07 14:24:51.210000
134819,290968,ZOTUSBBWIS,2019-04-01 12:44:00.000000,2019-10-07 14:24:51.210000
166846,321083,ZOTUSBBWIS,2019-04-30 15:14:59.999999,2019-10-07 14:24:51.210000
168433,338855,ZOTUSBBWIS,2019-05-27 11:32:00.000000,2019-10-07 14:24:51.210000
168986,350723,ZOTUSBBWIS,2019-06-05 17:51:00.000000,2019-10-07 14:24:51.210000
170387,386901,ZOTUSBBWIS,2019-07-09 17:13:00.000000,2019-10-07 14:24:51.210000
171004,406335,ZOTUSBBWIS,2019-07-25 18:42:00.000000,2019-10-07 14:24:51.210000
173632,510811,ZOTUSBBWIS,2019-10-07 14:25:00.000001,2019-10-07 14:24:51.210000


In [47]:
# Проверка размера выборки и количества уникальных ИНН в выборке

print('Shape: ', df_features.shape,'\n',
      'Nunique of inn: ', df_features.inn.nunique())

Shape:  (123927, 80) 
 Nunique of inn:  99921


In [48]:
# Проверка на количество входящих уникальных ИНН из размеченной выборки в выборку с заявками и признаками спарка

df_features[df_features.inn.isin(df_razmetka.inn.unique().tolist())].shape

(83999, 80)

In [49]:
df_razmetka.head()

,inn,data_otkrytija,data_zakrytija,year_mon_open,quarter_open,data_blokirovki,year_mon_block,diff_date_m,target
0,VWNMBTGADCEF,2018-06-18,2019-02-08,2018-06,2018Q2,NaT,None,999,0
1,EXSWZGFDABIU,2018-06-18,2018-06-21,2018-06,2018Q2,NaT,None,999,0
2,IUOMTGZHHT,2018-06-18,2019-01-09,2018-06,2018Q2,NaT,None,999,0
3,EUJAITITRDJL,2018-06-18,NaT,2018-06,2018Q2,NaT,None,999,0
4,TPNQVLMRXW,2018-06-18,NaT,2018-06,2018Q2,NaT,None,999,0


In [50]:
df_features.head()

,idinquiry,time_zajavki,tip_organizatsii,data_zajavki,year_mon_zajavki,quarter_zajavki,year_zajavki,inn,time_zagruzki_zajavki,kolichestvo_kompanij_s_analogichnym_direktorom_v_tom_zhe_regione,...,_kodokved,_prversokved,_a_upravljajuschaja_organizatsija_berem_dannye_iz_sektsii_svuprorg,_b_gendirektor_fl_,_uchrediteli_i_ih_izmenenie,_uchrediteli_inostrannye_kompanii,_uchrediteli_fizlitsa,istorija_smeny_soouchreditelej,data_zagruzki_zajavki,year_mon_zagruzki_zajavki
0,149105,2018-06-18 03:37:00.000000,ЮЛ,2018-06-18,2018-06,2018Q2,2018,BYKKBSYFTS,2018-06-18 03:34:03.993000,2.0,...,None,None,None,None,None,None,None,ЮЛ: 08.06.2018 0:00:00,2018-06-18,2018-06
2,149107,2018-06-18 03:49:00.000000,ИП,2018-06-18,2018-06,2018Q2,2018,MIWYERAPTINW,2018-06-18 03:46:46.243000,0.0,...,None,None,None,None,None,None,None,None,2018-06-18,2018-06
3,149109,2018-06-18 04:12:00.000000,ИП,2018-06-18,2018-06,2018Q2,2018,RXDNYNCXKXRF,2018-06-18 04:09:41.117000,0.0,...,None,None,None,None,None,None,None,None,2018-06-18,2018-06
4,149112,2018-06-18 04:56:00.000001,ЮЛ,2018-06-18,2018-06,2018Q2,2018,EGULYSKZPC,2018-06-18 04:53:47.150000,1.0,...,None,None,None,None,None,None,None,ЮЛ: 14.06.2018 0:00:00,2018-06-18,2018-06
10,149113,2018-06-18 05:19:00.000000,ИП,2018-06-18,2018-06,2018Q2,2018,RQAOYTOOVXCE,2018-08-06 06:07:58.430000,0.0,...,None,None,None,None,None,None,None,None,2018-08-06,2018-08


In [51]:
query = """
SELECT a.*,
    b.data_otkrytija,
    b.data_zakrytija,
    b.year_mon_open,
    b.quarter_open,
    b.data_blokirovki,
    b.year_mon_block,
    b.target
FROM df_features a
INNER JOIN df_razmetka b
ON a.inn=b.inn and a.data_zajavki<=b.data_otkrytija
ORDER BY a.data_zajavki
"""

df_final = ps.sqldf(query,locals()).drop_duplicates(subset=['idinquiry','inn'],keep='last')
df_final.shape

(82084, 87)

In [52]:
df_final[df_final.inn=='BQQAMCUYXH'][['idinquiry','inn','data_zajavki',
                                      'data_zagruzki_zajavki','data_otkrytija','data_blokirovki',
                                      'tip_organizatsii',
                                      'target']]

,idinquiry,inn,data_zajavki,data_zagruzki_zajavki,data_otkrytija,data_blokirovki,tip_organizatsii,target
19876,199413,BQQAMCUYXH,2018-10-26,2019-04-30,2019-04-30 00:00:00.000000,None,ЮЛ,0
20500,200877,BQQAMCUYXH,2018-10-30,2019-04-30,2019-04-30 00:00:00.000000,None,ЮЛ,0
21351,202812,BQQAMCUYXH,2018-11-01,2019-04-30,2019-04-30 00:00:00.000000,None,ЮЛ,0
23292,207130,BQQAMCUYXH,2018-11-12,2019-04-30,2019-04-30 00:00:00.000000,None,ЮЛ,0
25043,210966,BQQAMCUYXH,2018-11-19,2019-04-30,2019-04-30 00:00:00.000000,None,ЮЛ,0
25369,211714,BQQAMCUYXH,2018-11-20,2019-04-30,2019-04-30 00:00:00.000000,None,ЮЛ,0
27154,215626,BQQAMCUYXH,2018-11-27,2019-04-30,2019-04-30 00:00:00.000000,None,ЮЛ,0
28294,218227,BQQAMCUYXH,2018-12-03,2019-04-30,2019-04-30 00:00:00.000000,None,ЮЛ,0
30046,222261,BQQAMCUYXH,2018-12-10,2019-04-30,2019-04-30 00:00:00.000000,None,ЮЛ,0
30892,224018,BQQAMCUYXH,2018-12-12,2019-04-30,2019-04-30 00:00:00.000000,None,ЮЛ,0


In [53]:
df_final = df_final.drop_duplicates(subset=['inn','data_otkrytija'],keep='last')

In [54]:
# Проверка на то, что дата подачи заявки была позже даты загрузки/обработки заявки

df_final[df_final.data_zajavki>df_final.data_zagruzki_zajavki].head()

,idinquiry,time_zajavki,tip_organizatsii,data_zajavki,year_mon_zajavki,quarter_zajavki,year_zajavki,inn,time_zagruzki_zajavki,kolichestvo_kompanij_s_analogichnym_direktorom_v_tom_zhe_regione,...,istorija_smeny_soouchreditelej,data_zagruzki_zajavki,year_mon_zagruzki_zajavki,data_otkrytija,data_zakrytija,year_mon_open,quarter_open,data_blokirovki,year_mon_block,target


In [55]:
# Выведение таблицы по ИНН, датам и таргету для сравнения логики дат

df_final[['idinquiry',
'inn',
'data_zajavki',
'data_zagruzki_zajavki',
'data_otkrytija',
'data_zakrytija',
'data_blokirovki',
'target',
]].head()

,idinquiry,inn,data_zajavki,data_zagruzki_zajavki,data_otkrytija,data_zakrytija,data_blokirovki,target
0,149105,BYKKBSYFTS,2018-06-18,2018-06-18,2018-06-18 00:00:00.000000,None,None,0
1,149107,MIWYERAPTINW,2018-06-18,2018-06-18,2018-06-18 00:00:00.000000,None,None,0
2,149109,RXDNYNCXKXRF,2018-06-18,2018-06-18,2018-06-19 00:00:00.000000,None,None,0
3,149112,EGULYSKZPC,2018-06-18,2018-06-18,2018-06-21 00:00:00.000000,None,None,0
5,149115,MJULWZDIVP,2018-06-18,2018-06-18,2018-07-05 00:00:00.000000,None,2018-11-23 00:00:00.000000,1


In [57]:
# Имеет продукт или нет!

cred_prod = pd.read_excel('Наличие_кредитного_продукта.xlsx')
cred_prod['has_product'] = 1
print(cred_prod.shape)
cred_prod.head()

(9172, 2)


,inn,has_product
0,JIFDLDWCKC,1
1,OXUANITWYSYJ,1
2,IJWAMIETEASJ,1
3,VLUUKDPNJBTK,1
4,WIRLKQYSFY,1


In [58]:
df_final = df_final.merge(cred_prod,how='left',on='inn').\
drop_duplicates(subset=['inn','data_otkrytija'],keep='last')
df_final.head()

,idinquiry,time_zajavki,tip_organizatsii,data_zajavki,year_mon_zajavki,quarter_zajavki,year_zajavki,inn,time_zagruzki_zajavki,kolichestvo_kompanij_s_analogichnym_direktorom_v_tom_zhe_regione,...,data_zagruzki_zajavki,year_mon_zagruzki_zajavki,data_otkrytija,data_zakrytija,year_mon_open,quarter_open,data_blokirovki,year_mon_block,target,has_product
0,149105,2018-06-18 03:37:00.000000,ЮЛ,2018-06-18,2018-06,2018Q2,2018,BYKKBSYFTS,2018-06-18 03:34:03.993000,2.0,...,2018-06-18,2018-06,2018-06-18 00:00:00.000000,None,2018-06,2018Q2,None,None,0,NaN
1,149107,2018-06-18 03:49:00.000000,ИП,2018-06-18,2018-06,2018Q2,2018,MIWYERAPTINW,2018-06-18 03:46:46.243000,0.0,...,2018-06-18,2018-06,2018-06-18 00:00:00.000000,None,2018-06,2018Q2,None,None,0,NaN
2,149109,2018-06-18 04:12:00.000000,ИП,2018-06-18,2018-06,2018Q2,2018,RXDNYNCXKXRF,2018-06-18 04:09:41.117000,0.0,...,2018-06-18,2018-06,2018-06-19 00:00:00.000000,None,2018-06,2018Q2,None,None,0,NaN
3,149112,2018-06-18 04:56:00.000001,ЮЛ,2018-06-18,2018-06,2018Q2,2018,EGULYSKZPC,2018-06-18 04:53:47.150000,1.0,...,2018-06-18,2018-06,2018-06-21 00:00:00.000000,None,2018-06,2018Q2,None,None,0,NaN
4,149115,2018-06-18 05:29:00.000000,ЮЛ,2018-06-18,2018-06,2018Q2,2018,MJULWZDIVP,2018-06-18 05:27:15.587000,1.0,...,2018-06-18,2018-06,2018-07-05 00:00:00.000000,None,2018-07,2018Q3,2018-11-23 00:00:00.000000,2018-11,1,NaN


In [59]:
# Проверка на размер выборки и количества уникальных ИНН, входящих в неё

df_final.shape, df_final.inn.nunique()

((65061, 88), 65061)

In [60]:
# Создадим факторы на количество открытых/ закрытых счетов всего / в течение месяца/ в квартала по ИНН

cnt_open = df_open.drop_duplicates(subset=['inn','data_otkrytija']).\
groupby(by='inn',as_index=False).count().\
rename(columns={'data_otkrytija':'cnt_otkritija',
               'data_zakrytija':'cnt_zakrytija'})[['inn','cnt_otkritija','cnt_zakrytija']]

m_cnt_open = df_open.sort_values(by=['inn','data_otkrytija'],ascending=True).\
drop_duplicates(subset=['inn','data_otkrytija'],keep='last').\
groupby(by=['inn','year_mon_open'],as_index=False).count().\
rename(columns={'data_otkrytija':'m_cnt_otkritija',
               'data_zakrytija':'m_cnt_zakrytija'})[['inn','year_mon_open','m_cnt_otkritija','m_cnt_zakrytija']]

q_cnt_open = df_open.sort_values(by=['inn','data_otkrytija'],ascending=True).\
drop_duplicates(subset=['inn','data_otkrytija'],keep='last').\
groupby(by=['inn','quarter_open'],as_index=False).count().\
rename(columns={'data_otkrytija':'q_cnt_otkritija',
               'data_zakrytija':'q_cnt_zakrytija'})[['inn','quarter_open','q_cnt_otkritija','q_cnt_zakrytija']]

In [61]:
# Создадим факторы на количество заявок на открытие счетов всего / в течение месяца/ в квартала по ИНН

cnt_req = df_req.drop_duplicates(subset=['inn','data_zajavki']).\
groupby(by='inn',as_index=False).count().\
rename(columns={'data_zajavki':'cnt_zajavki'})[['inn','cnt_zajavki']]

m_cnt_req = df_req.sort_values(by=['inn','data_zajavki'],ascending=True).\
drop_duplicates(subset=['inn','data_zajavki'],keep='last').\
groupby(by=['inn','year_mon_zajavki'],as_index=False).count().\
rename(columns={'data_zajavki':'m_cnt_zajavki'})[['inn','year_mon_zajavki','m_cnt_zajavki']]

q_cnt_req = df_req.sort_values(by=['inn','data_zajavki'],ascending=True).\
drop_duplicates(subset=['inn','data_zajavki'],keep='last').\
groupby(by=['inn','quarter_zajavki'],as_index=False).count().\
rename(columns={'data_zajavki':'q_cnt_zajavki'})[['inn','quarter_zajavki','q_cnt_zajavki']]

In [62]:
# Присоедиянем сгенерированные признаки к выборке

df_final = df_final.merge(cnt_open,how='left',on='inn').\
merge(cnt_req,how='left',on='inn').\
drop_duplicates(subset=['inn','data_otkrytija'],keep='last')

df_final = df_final.merge(m_cnt_open,how='left',on=['inn','year_mon_open']).\
merge(m_cnt_req,how='left',on=['inn','year_mon_zajavki']).\
drop_duplicates(subset=['inn','data_otkrytija'],keep='last')

df_final = df_final.merge(q_cnt_open,how='left',on=['inn','quarter_open']).\
merge(q_cnt_req,how='left',on=['inn','quarter_zajavki']).\
drop_duplicates(subset=['inn','data_otkrytija'],keep='last')

In [63]:
# Проверка на размер выборки и количества уникальных ИНН, входящих в неё

df_final.shape, df_final.inn.nunique()

((65061, 97), 65061)

In [64]:
df_final.to_csv('df_final.csv',index=False)